<a href="https://colab.research.google.com/github/FabianWalocha/sentimentAnalysis/blob/master/CV_W2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CNN-LSTM approach

In [0]:
%cd /content
!rm -rf sentimentAnalysis
!git clone https://github.com/FabianWalocha/sentimentAnalysis.git
%cd sentimentAnalysis/

/content
Cloning into 'sentimentAnalysis'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 103 (delta 63), reused 63 (delta 30), pack-reused 0
Receiving objects: 100% (103/103), 6.28 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/sentimentAnalysis


In [0]:
%tensorflow_version 1.x
from sklearn import metrics
from random import shuffle
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC 
from keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
import  numpy as np
from matplotlib import pyplot as plt
import spacy
import os
import warnings
from CNNLSTM import score_CNN_LSTM as m1
from w2vLSTM import score_CNN_LSTM as m2
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
path_data = 'tweet_data.txt'

In [0]:
labeled_tweets = []
with open(path_data,'r', encoding='utf-8') as f:
    for tweet in f:
        t  = tweet.split(" ")
        labeled_tweets.append((t[0]," ".join(t[1:])))
shuffle(labeled_tweets)
tweets = [x for _,x in labeled_tweets]
labels = [y for y,_ in labeled_tweets]

In [0]:
print('Generating cross-folds...')
split_ratio = (0.8,0.1,0.1) #Train,Dev,Test
cross_folds = int((1-split_ratio[0])**(-1))
print(f'Using {cross_folds} crossfolds...')
assert sum(split_ratio) == 1
cross_fold_items = []
print('Training classifiers...')
for i in range(cross_folds):
    size = len(tweets)
    k1 = i*int(size*split_ratio[1])
    k2 = k1+int(size*split_ratio[1])
    k3 = k2+int(size*split_ratio[2])

    trainD = tweets[k3:] + tweets[:k1]
    testD = tweets[k1:k2]
    devD = tweets[k2:k3]
    print('Generating training data...')
    training_data = trainD
    training_labels = np.hstack(label for label in labels[k3:] + labels[:k1])
    print('Generating test data...')
    test_data = testD
    test_labels = np.hstack(label for label in labels[k1:k2])
    print('Generating dev data...')
    dev_data = devD
    dev_labels = np.hstack(label for label in labels[k2:k3])
    cross_fold_items.append((training_data,training_labels,test_data,test_labels,dev_data,dev_labels))
    assert len(test_data) + len(training_data) + len(dev_data) == size

Generating cross-folds...
Using 5 crossfolds...
Training classifiers...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...


In [0]:
print('Classifiers...')
lossT, accT, f1T, precT, recT = 0,0,0,0,0
res = np.zeros([5,5])
for i in range(cross_folds):
    print('#'*35)
    training_data,training_labels,test_data,test_labels,dev_data,dev_labels = cross_fold_items[i]
    loss, acc, f1, prec, rec = m2(training_data,training_labels,dev_data,dev_labels,test_data,test_labels,epochs=20, verbose=0)
    res[i,:] = [loss, acc, f1, prec, rec]

Classifiers...
###################################









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




5033/5033 [==============================] - 1s 263us/step
###################################
5033/5033 [==============================] - 2s 336us/step
###################################
5033/5033 [==============================] - 2s 416us/step
###################################
5033/5033 [==============================] - 2s 495us/step
###################################
5033/5033 [==============================] - 3s 588us/step


In [0]:
import pandas as pd

res_df = pd.DataFrame(data=np.round(res,3),columns=['loss','accuracy','f1_score','precision','recall'])
res_df

,loss,accuracy,f1_score,precision,recall
0,0.690,0.344,0.344,0.344,0.344
1,0.680,0.380,0.380,0.380,0.379
2,0.678,0.377,0.377,0.377,0.377
3,0.679,0.376,0.376,0.376,0.376
4,0.676,0.380,0.380,0.380,0.380


In [0]:
print(res_df.mean())
print()
print(res_df.std())

loss         0.6806
accuracy     0.3714
f1_score     0.3714
precision    0.3714
recall       0.3712
dtype: float64

loss         0.005459
accuracy     0.015421
f1_score     0.015421
precision    0.015421
recall       0.015287
dtype: float64
